In [1]:
from dataclasses import dataclass
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset
from pathlib import Path

In [2]:
data_dir = Path("/root/autodl-tmp/datasets--liuhaotian--LLaVA-CC3M-Pretrain-595K")

chat_file = Path(data_dir, "chat.json")

chat_data = pd.read_json(chat_file)
chat_data.shape


: 